In [23]:
import pandas as pd
import numpy as np
import random
from util import set_item, usage_masking, error_cal, error_masking, Processing, damage_sum, damage_percent, volume_checker, sampler, sampler_rate
from model import Kcluster, Let_model
import progressbar

In [24]:
df = pd.read_csv(r'G:\내 드라이브\work_space\Project_Files\BI공모전\분석과제 5\2023공공데이터활용BI공모전\2023공모전데이터.csv')

In [25]:
def model_checker(item_list, Dataframe):
    medium_volume = 0
    small_volume = 0
    
    for i in item_list:
        power_volume = set_item(i, Dataframe)[0]
        
        if power_volume > 1000:
            medium_volume = medium_volume + power_volume
        else:
            small_volume = small_volume + power_volume
    
    total_volume = medium_volume + small_volume
    
    return total_volume, medium_volume, small_volume

In [26]:
def money_sum(item_list, Dataframe):
    total_real_array = np.zeros((31, 24))
    total_pred_array = np.zeros((31, 24))
    total_power_volumes = 0

    for i in item_list:
        temp = Processing(i, Dataframe)
        total_real_array = total_real_array + temp.real_array
        total_pred_array = total_pred_array + temp.pred_array
        total_power_volumes = total_power_volumes + temp.power_volume
        
    total_usage_masked = usage_masking(total_real_array / total_power_volumes) 
    total_error_array = error_cal(total_power_volumes, total_real_array, total_pred_array)
    total_error_masked = error_masking(error_cal(total_power_volumes, total_real_array, total_pred_array))
    total_real_money = total_real_array * total_usage_masked * total_error_masked
    total_best_money = total_real_array * 4
    total_take_percent = (total_real_money.sum()) / (total_best_money.sum())
    
    return total_real_money.sum(), total_best_money.sum(), total_take_percent

In [ ]:
Power_list = list(df.Power_Code.unique())
model = Kcluster(Power_list, df)

for num in range(100):
    random_k = random.randint(4, 9)
    random_first_search = random.randint(30, 200)
    random_min_length = random.randint(3, 12)
    random_max_volume = (197799 / random_k) * (random.uniform(0.98, 1.3))

    result = Let_model(Power_list, model, random_k, random_first_search, random_min_length, random_max_volume, df)
    check_point = 0
    for i in result:
        total, medium, small = model_checker(i, df)
        if total >= 20000:
            check_point = check_point + 1
        if small >= medium:
            check_point = check_point + 1

    if check_point == 2*random_k:
        print('Collect Result')
        
        temp_df = pd.DataFrame()
        for i, value in enumerate(result):
            real_money, best_money, percent = money_sum(value, df)
            temp_df.loc[i, 'item_complex'] = value
            temp_df.loc[i, 'Total_volume'] = model_checker(value, df)
            temp_df.loc[i, 'Total_Money'] = real_money * 1000
            temp_df.loc[i, 'Total_Best_Money'] = best_money * 1000
            temp_df.loc[i, 'Money_percent'] = percent
    
        temp_df.to_csv(f'./result_{num}.csv', index=False)

In [ ]:
Power_list = list(df.Power_Code.unique())

a = 0
for j, i in enumerate(Power_list):
    temp = Processing(i, df)
    a= a+temp.power_volume
    print(j)

print(a)
